# Pymaceuticals Inc.
---

Analysis:

Ramicane and Capomulin as treatments were successful in reducing the volume of the tumor size during the trial period. Run

Ramicane and Capomulin also had a smaller Standard Deviation of their tumor size. This shows that the regimens produced more consistent results among the mice that they tested. The higher Standard Deviation numbers of the other regimens show a more sporadic result range amongst the mice in those groups.

In the Capomulin study, the mouse size and tumor mass coorelated.  More study would be needed to see if the weight of the tumor is what made the mouse "larger" or if bigger mice grew bigger tumors.

T

In [1]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import scipy.stats as st

# Study data files
mouse_metadata_path = "./data/Mouse_metadata.csv"
study_results_path = "./data/Study_results.csv"

# Constants
MOUSE_ID = "Mouse ID"
TIME_POINT = "Timepoint"
TUMOR_VOLUME = "Tumor Volume (mm3)"
WEIGHT_G = "Weight (g)"


# Read the mouse data and the study results
mouse_metadata = pd.read_csv(mouse_metadata_path)
study_results = pd.read_csv(study_results_path)

# Combine the data into a single dataset
science_study_data_complete_df = pd.merge(study_results, mouse_metadata, how= "left", on = MOUSE_ID)
# Display the data table for preview
science_study_data_complete_df





FileNotFoundError: [Errno 2] No such file or directory: './data/Mouse_metadata.csv'

In [ ]:
# Checking the number of mice.
len(science_study_data_complete_df [MOUSE_ID].unique())



In [ ]:
# Getting the duplicate mice by ID number that shows up for Mouse ID and Timepoint. 
duplicate_mouse_id=science_study_data_complete_df [science_study_data_complete_df.duplicated(subset =["Mouse ID","Timepoint"])]["Mouse ID"].unique()
duplicate_mouse_id


In [ ]:
# Optional: Get all the data for the duplicate mouse ID. (by searching thru the df for the particular ID)
duplicated_mouse_dataset = science_study_data_complete_df[science_study_data_complete_df["Mouse ID"] == duplicate_mouse_id[0]]
duplicated_mouse_dataset

In [ ]:
clean_study_data_complete = science_study_data_complete_df[science_study_data_complete_df['Mouse ID'].isin(duplicate_mouse_id)==False]
clean_study_data_complete









In [ ]:
# Checking the number of mice in the clean DataFrame.
len(clean_study_data_complete["Mouse ID"].unique())


## Summary Statistics

In [ ]:
# Generate a summary statistics table of mean, median, variance, standard deviation, and SEM of the tumor volume for each regimen

# Use groupby and summary statistical methods to calculate the following properties of each drug regimen: 
# mean, median, variance, standard deviation, and SEM of the tumor volume. 
means = clean_study_data_complete.groupby("Drug Regimen").mean()[TUMOR_VOLUME]
medians = clean_study_data_complete.groupby("Drug Regimen").median()[TUMOR_VOLUME]
variances = clean_study_data_complete.groupby("Drug Regimen").var()[TUMOR_VOLUME]
standard_deviation = clean_study_data_complete.groupby("Drug Regimen").std()[TUMOR_VOLUME]
sems =clean_study_data_complete.groupby("Drug Regimen").sem()["Tumor Volume (mm3)"]

summary_table = pd.DataFrame({
    "Mean Tumor Volume": means,
    "Median Tumor Volume": medians,
    "Tumor Volume Variance": variances,
    "Tumor Volume Std. Dev": standard_deviation,
    "Tumor Volume Std. Err": sems
})
summary_table

In [ ]:
# Generate a summary statistics table of mean, median, variance, standard deviation, 
# and SEM of the tumor volume for each regimen

# Using the aggregation method, produce the same summary statistics in a single line.
summary_table = clean_study_data_complete.groupby("Drug Regimen").agg({"Tumor Volume (mm3)" : ["mean","median", "var", "std", "sem"]
})
summary_table

## Bar and Pie Charts

In [ ]:
# Generate a bar plot showing the total number of timepoints for all mice tested for each drug regimen using Pandas.
counts = clean_study_data_complete["Drug Regimen"].value_counts()
counts.plot(kind= "bar")
plt.xlabel("Drug Regimen")
plt.xticks(rotation = 90)
plt.ylabel("Number of Mice Tested")
plt.show()

In [ ]:
# Generate a bar plot showing the total number of timepoints for all mice tested for each drug regimen using pyplot.
counts = clean_study_data_complete["Drug Regimen"].value_counts()
plt.bar(counts.index.values, counts.values)
plt.xlabel("Drug Regimen")
plt.xticks(rotation = 90)
plt.ylabel("Number of Mice Tested")
plt.show()

In [ ]:
# Generate a pie plot showing the distribution of female versus male mice using Pandas
counts = clean_study_data_complete.Sex.value_counts()
counts.plot(kind= "pie",autopct = "%1.1f%%")


In [ ]:
# Generate a pie plot showing the distribution of female versus male mice using pyplot
counts = clean_study_data_complete["Sex"].value_counts()
plt.pie(counts.values, labels=counts.index.values, autopct="%1.1f%%")
plt.ylabel("Sex")
plt.show

## Quartiles, Outliers and Boxplots

In [ ]:
# Calculate the final tumor volume of each mouse across four of the treatment regimens:  
# Capomulin, Ramicane, Infubinol, and Ceftamin

# Start by getting the last (greatest) timepoint for each mouse

max_tumor = clean_study_data_complete.groupby(["Mouse ID"])["Timepoint"].max()
max_tumor = max_tumor.reset_index()

# Merge this group df with the original DataFrame to get the tumor volume at the last timepoint
merged_data = max_tumor.merge(clean_study_data_complete, on =["Mouse ID","Timepoint"], how="left")

In [ ]:
# Put treatments into a list for for loop (and later for plot labels)
treatment_list = ["Capomulin", "Ramicane","Infubinol", "Ceftamin", "Stelasyn", "Propiva", "Naftisol"]
# Create empty list to fill with tumor vol data (for plotting)

tumor_vol_list = []
# Calculate the IQR and quantitatively determine if there are any potential outliers. 
for drug in treatment_list:

    # Locate the rows which contain mice on each drug and get the tumor volumes
    final_tumor_volume = (merged_data.loc[merged_data["Drug Regimen"] == drug, "Tumor Volume (mm3)"])
    
    # add subset  -
    
    tumor_vol_list.append(final_tumor_volume)
    # Determine outliers using upper and lower bounds
    quartiles = final_tumor_volume.quantile([.25,.5,.75])
    lowerq = quartiles[0.25]
    upperq = quartiles[0.75]
    iqr =upperq - lowerq 
    lowerbound = lowerq - (1.5 * iqr)
    upperbound = upperq + (1.5 * iqr)
    outliers = final_tumor_volume.loc[(final_tumor_volume < lowerbound)] 
    print(f" {drug}'s potential outliers {outliers}")


In [ ]:
# Generate a box plot that shows the distrubution of the tumor volume for each treatment group.
orange_out = dict(markerfacecolor = "red", markersize = 20)
plt.boxplot(tumor_vol_list, labels = treatment_list, flierprops = orange_out)
plt.ylabel("Final Tumor Volume (mm3)")
plt.show()

## Line and Scatter Plots

In [ ]:
# Generate a line plot of tumor volume vs. time point for a mouse treated with Capomulin
capomulin_table = clean_study_data_complete[clean_study_data_complete["Drug Regimen"]=="Capomulin"]
mousedata = capomulin_table[capomulin_table["Mouse ID"]== "l509" ]
plt.plot(mousedata["Timepoint"], mousedata["Tumor Volume (mm3)"])
plt.xlabel("Timepoint (days")
plt.ylabel("Tumor Volume (mm3)")
plt.title("Capomulin Treatment of Mouse")

In [ ]:
# Generate a scatter plot of average tumor volume vs. mouse weight for the Capomulin regimen
capomulin_table = clean_study_data_complete[clean_study_data_complete["Drug Regimen"]=="Capomulin"]
capomulin_average = capomulin_table.groupby(["Mouse ID"]).mean()
plt.scatter(capomulin_average["Weight (g)"], capomulin_average["Tumor Volume (mm3)"])
plt.xlabel("Weight(g)")
plt.ylabel("Average Tumor Volume")
plt.show()



## Correlation and Regression

In [ ]:
# Calculate the correlation coefficient and linear regression model 
# for mouse weight and averagetumor volume for the Capomulin regimen
corr = st.pearsonr(capomulin_average["Weight (g)"], capomulin_average["Tumor Volume (mm3)"])
print (f" The coorelation between mouse weight and the average mouse tumor volume is {round(corr[0],2)}")
model = st.linregress(capomulin_average["Weight (g)"], capomulin_average["Tumor Volume (mm3)"])
slope = model[0]
b= model[1]
y_value =capomulin_average["Weight (g)"] *slope + b
plt.scatter(capomulin_average["Weight (g)"], capomulin_average["Tumor Volume (mm3)"])
plt.plot(capomulin_average["Weight (g)"], y_value,color = "red")
plt.xlabel("Weight (g)")
plt.ylabel("Average Tumor Volume (mm3)")

